In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(0, '/home/ektov-av/notebooks/myscripts/support_library/') 
sys.path.insert(0, '/home/ektov-av/python35-libs/lib/python3.5/site-packages/')
sys.path.insert(0, '/home/ektov-av/notebooks/labdata/lib/')
from spark_connector import SparkConnector
from sparkdb_loader import spark
from sshloader import Ssh
from time import sleep

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [ ]:
for i in range(9,1000):
    try:
        print('working with: februs{}.ca.sbrf.ru'.format(str(i)))
        _ssh = Ssh(host='februs{}.ca.sbrf.ru'.format(str(i)))
        _ssh.setup()

        file_name  = 'pip.conf'
        remotepath = Path('./').as_posix()
        localpath  = Path('/home/ektov-av/pip').as_posix()
        remdir = 'pip'
        _ssh.putFiles(localpath, remotepath, file_name, destFolderName=remdir)

        bash_profile  = '.bash_profile'
        remotepath = Path('./').as_posix()
        localpath  = Path('/home/ektov-av/').as_posix()
        remdir = ''
        _ssh.putFiles(localpath, remotepath, bash_profile, destFolderName=remdir)

        sleep(1)

        stdin,stdout,stderr=_ssh.ssh.exec_command('cd /home/ektov-av; source {bash_profile}'.format(bash_profile=bash_profile))
        outlines=stdout.readlines()  
        resp=''.join(outlines)

        bootstrap  = 'bootstrap_py35_data_laboratory_februs.sh'
        remotepath = Path('./').as_posix()
        localpath  = Path('/home/ektov-av/').as_posix()
        remdir = ''
        _ssh.putFiles(localpath, remotepath, bootstrap, destFolderName=remdir)

        stdin,stdout,stderr=_ssh.ssh.exec_command('cd /home/ektov-av; chmod u+x ./{bootstrap}; ./{bootstrap}'.format(bootstrap=bootstrap))
        outlines=stdout.readlines()  
        resp=''.join(outlines)

        remdir = 'python35-libs/lib/python3.5/site-packages/'
        stdin,stdout,stderr=_ssh.ssh.exec_command('cd /home/ektov-av; mkdir -p {__dir}'.format(__dir=remdir))
        outlines=stdout.readlines()  
        resp=''.join(outlines)

        pythondir = '/home/ektov-av/python35-libs/lib/python3.5/site-packages/'
        stdin,stdout,stderr=_ssh.ssh.exec_command('cd /home/ektov-av; source .bash_profile; pip35 install yargy -t {__dir}'.format(__dir=pythondir))
        outlines=stdout.readlines()  
        resp=''.join(outlines)
    except:
        pass

In [ ]:
# remdir = '/home/ektov-av/python35-libs/lib/python3.5/site-packages/'
# _ssh.putFiles(localpath, remotepath, file_name, destFolderName=remdir)

In [ ]:
# remotepath = Path('./data/').as_posix()
# localpath  = Path.joinpath(Path(os.getcwd()),'csv').as_posix()
# _ssh.getFiles(remotepath, localpath, file_name)

In [ ]:
# file_name  = 'avito_db_nedvizhimost_04.15.2019.csv'
# remotepath = Path.joinpath(Path('/mnt/data/avito_parse/get/csv/'),file_name).as_posix()
# localpath  = Path.joinpath(Path(os.getcwd()),'csv', file_name).as_posix()
# sftp.get(remotepath, localpath)
# sftp.close()

### SPARK CONNECTOR

In [4]:
sp = spark(schema='t_team_speech_analytics')

dynamic_allocation is ON


In [5]:
hive = sp.sql
transcriptions = hive.table('t_team_speech_analytics.telemarketing')

In [ ]:
transcriptions.printSchema()

In [7]:
df = transcriptions.limit(4).show()
df

,Dialogue_ID,Date,duration,Channel_0,Channel_1,Text,nvcswitchagentids,nvcdialednumbers
0,6605395277561004793,2018-09-26 05:07:00,73,а. здравствуйте. да слушаю вас. я. я отвечаю. ...,добрый день. меня зовут ольга я специалист сбе...,Канал 0: а\nКанал 1: добрый день\nКанал 0: здр...,"[4156, 54638]",[89174079449]
1,6605400783709078672,2018-09-26 05:28:00,36,аллё здравствуйте. нет. знаком а вы по какому ...,добрый день. меня зовут алексей я специалист с...,Канал 1: добрый день\nКанал 0: аллё здравствуй...,"[3999, 54135]",[89133601454]
2,6605405409388856779,2018-09-26 05:46:00,5,а. а. аллё. да да. да нет меня не интересует. ...,добрый день. добрый день а сбербанк специалист...,Канал 0: а\nКанал 1: добрый день\nКанал 0: а\n...,"[2541, 52016]",[89098237581]
3,6605405598367417813,2018-09-26 05:47:00,50,добрый. да. сергей. девушка давайте а потом я ...,добрый день. меня я специалист сбербанка. явля...,Канал 1: добрый день\nКанал 0: добрый\nКанал 1...,"[54328, 4044]",[89227948043]


In [ ]:
transcriptions_text = hive.table('t_team_speech_analytics.telemarketing') \
                .select('Dialogue_ID', 'Text')

In [ ]:
import os
import shlex
import subprocess

def install_package(x):
    import sys
    sys.path.insert(0, '/home/ektov-av/python35-libs/lib/python3.5/site-packages/')
    import yargy
#     out, err = proc.communicate()
    return yargy.__file__
            
# def install_package(x):
#     out = subprocess.check_call('/opt/cloudera/parcels/PYENV.GPUAI-3.6.pyenv.p0.1/bin/pip3 install yargy ' \
#         '--user --index-url=http://mirror.ca.sbrf.ru/pypi/simple/ '\
#         '--extra-index-url=http://mirror.ca.sbrf.ru/pypi_old/simple '\
#         '--trusted-host=mirror.ca.sbrf.ru', shell=True, stderr=subprocess.PIPE)
#     return out

rdd = sp.sc.parallelize(list(range(sp.sc._jsc.sc().getExecutorMemoryStatus().size())))
output = rdd.map(install_package).collect()
output

In [ ]:
res = sp.get(input='news_monitoring_news_with_events', where='limit 100')

In [ ]:
res.show()